### HW2b by Jordan Gitleman

We can continue the previous project as we explained in class by using a feature detector instead of manually picking the points. 

1.	Use a Harris corner or a SIFT feature detector and then match those points using the brute force matcher. How many points would you pick? \
(a)	Use SVD to compute a homography on the overconstrained linear equations that the feature points imply. Remake the mosaic. \
(b)	Now use the cv2.homography with Ransac on the src and dst points after you match the feature points and remake the mosaic. 
